##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Time windows

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l08c04_time_windows.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l08c04_time_windows.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Setup

In [1]:
import tensorflow as tf

## Time Windows

First, we will train a model to forecast the next step given the previous 20 steps, therefore, we need to create a dataset of 20-step windows for training.

In [2]:
#0 ~ 9까지의 값을 갖는 dataset 생성하기

dataset = tf.data.Dataset.range(10)

#val.numpy()를 쓰지 않고, 그냥 val만 프린트하면 배열이 반환됨!(텐서가 반환됨)
for val in dataset:
    print(val.numpy())

0
1
2
3
4
5
6
7
8
9


In [3]:
dataset = tf.data.Dataset.range(10)

#크기가 5이고, 1씩 옆으로 가는 윈도우 만들기
dataset = dataset.window(5, shift=1)

#윈도우 1개는 데이터셋 1개에 해당! -> 윈도우데이터셋 안에서 같은 방식으로 출력 가능
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 
6 7 8 9 
7 8 9 
8 9 
9 


In [4]:
dataset = tf.data.Dataset.range(10)

#크기가 5가 아닌 윈도우데이터셋은 버림 (머신러닝 모델 일부는 정해진 인풋 사이즈가 필요하기 때문)
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window_dataset in dataset:
    for val in window_dataset:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


In [5]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)

#데이터셋 안에 윈도우 데이터셋이 여러 개 있는 구조가 아니라, 데이터셋 1개 내부에 배열 여러개가 있는 구조를 만들고 싶기 때문에 아래와 같이 작성함!
# flat_map : 그냥 map과 비슷함! dataset 안의 모든 'window'에 대해 괄호 내부 함수(window.batch(5))를 적용시킴
# window.batch(5) : 각 윈도우에서 5개의 값을 가져와서 배열(텐서) 1개로 만들어줌
#끝나면, 텐서 5개를 갖고 있는 데이터셋이 만들어짐!
dataset = dataset.flat_map(lambda window: window.batch(5))
for window in dataset:
    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


In [6]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))

#머신러닝에선 input_feature랑 label(예측할 결과값)이 구분되어야 하기 때문에, 각 배열을 두 개로 쪼개는 코드가 필요함
#map으로 lambda함수를 모든 윈도우배열에 적용시킴
#lambda = 배열 두 개 반환: 각 윈도우배열을 0 ~ 마지막 -1, 마지막 값만 갖는 배열 2개로 쪼갬
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
for x, y in dataset:
    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


In [7]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))

#윈도우배열을 섞어주기 - gradient descent로 weight를 조정하는 모델에는 필수적인 과정!
dataset = dataset.shuffle(buffer_size=10)
for x, y in dataset:
    print(x.numpy(), y.numpy())

[4 5 6 7] [8]
[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[5 6 7 8] [9]


In [9]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)

#윈도우배열을 2개씩 묶어 새로운 배열 만들기
#prefetch: 텐서플로우가 1개의 데이터를 다루는 동안 그 다음 데이터를 미리 불러오게 하는 것! 실행 시간을 줄임
dataset = dataset.batch(2).prefetch(1)
for x, y in dataset:
    print("x =", x.numpy())
    print("y =", y.numpy())

x = [[0 1 2 3]
 [1 2 3 4]]
y = [[4]
 [5]]
x = [[2 3 4 5]
 [4 5 6 7]]
y = [[6]
 [8]]
x = [[3 4 5 6]
 [5 6 7 8]]
y = [[7]
 [9]]


In [10]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset